In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [54]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [ ]:
df_vs

# hier ein Test von Barnaby, wie man die Namen alleine mit "Split" separieren kann

- Vielleicht zuerst noch mit Pandas nur die Vorstösse raussuchen. Ist ja das, was mich interessiert.
- Rausfinden, wie ich die Namen nun rausziehe und in ein aufklappbares Dataframe stelle


In [ ]:

def splitting(elem):
    elem = elem.split('betreffend')[0].strip()
    if 'von' in elem:
        elem = elem.split('von')[1].strip()
        elem = elem.replace(' und', ',')
    elif 'der' in elem:
        elem = elem.split('der')[1].strip()    
    return elem
df_vs["Geschäft"]=df_vs['Geschäft'].apply(splitting) #.apply ist um eine Funktion auf eine ganze Reihe Pandas einzusetze

# Nun rausfinden, wie ich ein DataFrame machen kann, das man aufklappt

In [ ]:
df_vs.head()

In [ ]:
#das funktionerit also nicht
def splitkomma(elem):
    elem=elem.split(", ")
    return elem


In [ ]:
df_vs["Geschäft"]=df_vs["Geschäft"].apply(splitkomma) 

In [ ]:
# Hier die Lösung, wie ich für jeden Namen ein einzelnes Geschäft erhalte

In [ ]:
df_ex=df_vs.explode('Geschäft').reset_index(drop=True)

In [ ]:
df_ex.rename(columns={'Geschäft':'Namenganz'}, inplace=True)

In [ ]:
df_ex.to_csv("KRFrames/df_ex")

In [ ]:
df_ex=pd.read_csv("KRFrames/df_ex")

In [ ]:
# hier schaue ich, wie gross das Dataframe ohne die Kommissionen und so ist.
df_ex_cl=df_ex[df_ex["Namenganz"].str.contains("Kommission|kommission")==False]

In [ ]:
df_ex_cl=df_ex_cl[["VorlageNR", "Namenganz", ]]

In [ ]:
# ICH bereinige von Hand im Excel
df_ex_cl.to_excel("KRFrames/df_ex_cl.xlsx")

In [ ]:
df_ex_cl[df_ex_cl["Namenganz"].str.contains("zur")]
#oook, die Alternativen wurden unteschiedliche benamselt. Das muss ich genauer anschauen.
# Auch andere Parteien werden zum Teil doppelt genannt. 

# Bereinigen der Parteinamen
Diese werden in mindestens drei unterschiedlichen Arten genannt

### Ich habe mich fürs Bereinigen im Excel entschieden, da Daten noch übersichtlich sind

In [43]:
# Hier nun das im Excel bereinigte Sheet
df_ex_ber=pd.read_excel("KRFrames/df_ex_cl_geputzt.xlsx")

### Problem erkannt: 
offenbar werden gekoppelte Namen in den Geschäften nur mit den ersten aufgeführt.

In [44]:
df_ex_ber.set_index("Unnamed: 0")


,VorlageNR,Namenganz
Unnamed: 0,,
0,3020,Luzian Franzini
1,3020,Esther Haas
2,3019,Luzian Franzini
3,3019,Rita Hofer
4,3019,Tabea Zimmermann Gibson
...,...,...
1937,836,Käty Hofer Buser
1940,810,Marcel Meyer
1941,801,Hans Abicht


In [ ]:
df_ex_ber.shape

In [45]:
# Nun das Frame mit den Namen einlesen
df_np=pd.read_excel('KRFrames/Alle_Namen_mit_Allianz_und_ohne_ergänzt.xlsx')

In [46]:
#von der Bearbeitung hat es nun noch einen anderen Index. stört uns nicht - Wir entfernen "unnamed:0"
del df_np["Unnamed: 0"]

In [41]:
df_np.shape #voila

(410, 3)

In [ ]:
#Nun die beiden zusammenfügen - an den Namen
df_alles=pd.merge(df_np, df_ex_ber, how="inner",on= "Namenganz", )

In [ ]:
df_alles=df_alles.sort_values("Unnamed: 0")

In [ ]:
df_alles

In [ ]:
df_alles.sort_values("P").tail(41)# So, es sind noch einige wenige, die offenbar fehlerhaft sind
# Da es nur noch einzelne Positionen betrifft, finde ich es so jetzt ok.
                            

In [ ]:
df_alles.set_index("Unnamed: 0")
del df_alles["Unnamed: 0.1"]
# nun das df_alles noch abspeichern für den Endcode
df_alles.to_csv("KRFrames/Frame_mit_Namen_und_Parteien")

In [ ]:
df_alles.dtypes

In [ ]:
del df_alles["Unnamed: 0.1"]


In [ ]:
# nun das df_alles noch abspeichern für den Endcode
df_alles.to_csv("KRFrames/Frame_mit_Namen_und_Parteien")

# Bereinigung, Parteien und Doppelnamen
offenbar werden bei mehrern Parteien diese nicht richtig getrennt. ich werde diese nun:
- dataframes mit str.contains machen und die Parteien rausfiltern und sie richtig benennen
- dazu sage ich allen jeweiligen Spalten, sie sollen nun z.b. SVP-Fraktion heissen.
- Fäür jede Patei inkl Vorlagenummer ein neues Dataframe abspeichern
- alle an die Namensliste hänen
- Dublikate bei Namen und Vorlagenummern löschen

In [22]:
df_alles1=pd.read_csv("KRFrames/Frame_mit_Namen_und_Parteien")

In [4]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [5]:
df_vs1=df_vs[["VorlageNR", "Geschäft"]]

In [6]:
#die Dataframes
df_alg=df_vs1[df_vs1["Geschäft"].str.contains("Alternative")]
df_sp=df_vs1[df_vs1["Geschäft"].str.contains("SP")]
df_cvp=df_vs1[df_vs1["Geschäft"].str.contains("CVP")]
df_fdp=df_vs1[df_vs1["Geschäft"].str.contains("FDP")]
df_glp=df_vs1[df_vs1["Geschäft"].str.contains("GLP")]
df_svp=df_vs1[df_vs1["Geschäft"].str.contains("SVP")]

In [10]:
#hier eine Funktion, um bei allen Namen die jeweilige Partei einzufügen.
def pareiersetz(satz):
    satz=partei
    return satz        

In [19]:
partei="SVP-Fraktion"
df_svp["Geschäft"]=df_svp["Geschäft"].apply(pareiersetz)
partei="FDP-Fraktion"
df_fdp["Geschäft"]=df_fdp["Geschäft"].apply(pareiersetz)
partei="CVP-Fraktion"
df_cvp["Geschäft"]=df_cvp["Geschäft"].apply(pareiersetz)
partei="GLP-Fraktion"
df_glp["Geschäft"]=df_glp["Geschäft"].apply(pareiersetz)
partei="SP-Fraktion"
df_sp["Geschäft"]=df_sp["Geschäft"].apply(pareiersetz)
partei="Alternative"
df_alg["Geschäft"]=df_alg["Geschäft"].apply(pareiersetz)

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [27]:
#zuerst die Parteiframes zusammenfügen
frames=[df_svp, df_fdp, df_glp, df_cvp, df_sp, df_alg]
df_alpart = pd.concat(frames,sort=True)


In [29]:
#nun den Namen dem Frame Namenganz anpassen
df_alpart.rename(columns={"Geschäft":"Namenganz"}, inplace=True)

In [47]:
# nun muss ich das dataframe noch mit jenem der Parteien zusammenfügen
df_alpart1=pd.merge(df_alpart, df_np, how="inner",on= "Namenganz")

In [48]:
df_alpart1

,Namenganz,VorlageNR,P,Unnamed: 0.1
0,SVP-Fraktion,3014,SVP,NaN
1,SVP-Fraktion,2977,SVP,NaN
2,SVP-Fraktion,2955,SVP,NaN
3,SVP-Fraktion,2941,SVP,NaN
4,SVP-Fraktion,2937,SVP,NaN
...,...,...,...,...
296,SP-Fraktion,1253,SP,NaN
297,SP-Fraktion,1211,SP,NaN
298,SP-Fraktion,1188,SP,NaN
299,SP-Fraktion,1187,SP,NaN


In [49]:
#nun mit df_alles zusammenfügen
newframes=[df_alles1, df_alpart1]
df_alles2=pd.concat(newframes, sort=True)

In [50]:
df_alles2

,Namenganz,P,Unnamed: 0,Unnamed: 0.1,VorlageNR
0,Luzian Franzini,ALG,1828.0,0.0,3020
1,Esther Haas,ALG,1018.0,1.0,3020
2,Luzian Franzini,ALG,1829.0,2.0,3019
3,Rita Hofer,ALG,1231.0,3.0,3019
4,Tabea Zimmermann Gibson,ALG,1323.0,4.0,3019
...,...,...,...,...,...
296,SP-Fraktion,SP,NaN,NaN,1253
297,SP-Fraktion,SP,NaN,NaN,1211
298,SP-Fraktion,SP,NaN,NaN,1188
299,SP-Fraktion,SP,NaN,NaN,1187


In [51]:
df_allesgut=df_alles2.drop_duplicates(subset=['VorlageNR', 'Namenganz'], keep=False)

In [52]:
df_allesgut.groupby("VorlageNR").count()

,Namenganz,P,Unnamed: 0,Unnamed: 0.1
VorlageNR,,,,
282,1,1,1,1
304,1,1,1,1
801,1,1,1,1
810,1,1,1,1
836,1,1,1,1
...,...,...,...,...
3010,4,4,4,4
3017,3,3,3,3
3018,3,3,3,3


In [53]:
df_allesgut.to_csv("KRFrames/Frame_Namen_Partei_endversion")